In [67]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from tqdm.auto import tqdm
import networkx as nx
import heapq

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

### Utility Functions

In [68]:
def load_data():
    global CITIES, DIST_MATRIX, median
    CITIES = pd.read_csv('italy.csv', header=None, names=['name', 'lat', 'lon']) # contains all cities with its lat and lon
    DIST_MATRIX = np.zeros((len(CITIES), len(CITIES))) # quadratice matrix of distances between cities, initialized with all zeros
    for c1, c2 in combinations(CITIES.itertuples(), 2): # for each pair of cities
        DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
            (c1.lat, c1.lon), (c2.lat, c2.lon)
        ).km # compute the distance between lat and lon
    median = np.median(DIST_MATRIX.reshape(1, -1))
    DIST_MATRIX[DIST_MATRIX > median] = np.inf # keeping all the distances that are lower than the median, setting to inf all the distances higher
    

In [69]:
def path_cost(path):
    tot_cost = 0
    for c1, c2 in zip(path, path[1:]):  # per ogni coppia di città consecutive
        tot_cost += DIST_MATRIX[c1, c2]  # somma la distanza tra le due città
    return tot_cost

### Networkx shortest_path method to check the Dijkstra correctness

In [70]:
load_data()
G = nx.from_numpy_array(DIST_MATRIX, create_using=nx.Graph())

s_city = 'Rome'
e_city = 'Bolzano'

start_city = CITIES[CITIES['name'] == s_city].index[0]
end_city = CITIES[CITIES['name'] == e_city].index[0]

try:
    shortest_path = nx.shortest_path(G, source=start_city, target=end_city, weight='weight')
    path_cities = CITIES.iloc[shortest_path]['name'].tolist()
    print(f"--------- Looking for path between {s_city} and {e_city} ---------")
    print("Best path:", " -> ".join(path_cities))
    print("Cost for best path: ", path_cost(shortest_path))
except nx.NetworkXNoPath:
    print("There is not a path between the two cities!")


--------- Looking for path between Rome and Bolzano ---------
Best path: Rome -> Forlì -> Bolzano
Cost for best path:  520.5980314423081


### Dijkstra with angle limit

In [71]:
# Compute angle between two points
def compute_angle(pointA, pointB):
    lat1, lon1 = np.radians(pointA)
    lat2, lon2 = np.radians(pointB)
    d_lon = lon2 - lon1
    x = np.sin(d_lon) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - (np.sin(lat1) * np.cos(lat2) * np.cos(d_lon))
    initial_bearing = np.arctan2(x, y)
    initial_bearing = np.degrees(initial_bearing)
    return (initial_bearing + 360) % 360

# Fixed Dijkstra function with angle limit
def dijkstra_with_angle_limit(start, end, matrix=DIST_MATRIX, cities=CITIES, max_angle_deviation=30):
    num_nodes = matrix.shape[0]
    dist = np.full(num_nodes, np.inf)
    dist[start] = 0
    prev = np.full(num_nodes, None)
    queue = [(0, start)]
    
    # Compute the angle between two cities
    start_coords = (cities.iloc[start]['lat'], cities.iloc[start]['lon'])
    end_coords = (cities.iloc[end]['lat'], cities.iloc[end]['lon'])
    main_bearing = compute_angle(start_coords, end_coords)

    while queue:
        current_dist, u = heapq.heappop(queue)
        if u == end:
            break
        if current_dist > dist[u]:
            continue

        # Looking the nearby cities
        for v in range(num_nodes):
            if matrix[u, v] != np.inf:
                u_coords = (cities.iloc[u]['lat'], cities.iloc[u]['lon'])
                v_coords = (cities.iloc[v]['lat'], cities.iloc[v]['lon'])
                bearing_uv = compute_angle(u_coords, v_coords)
                
                # Checking the angle deviation
                if abs(bearing_uv - main_bearing) <= max_angle_deviation:
                    alt = current_dist + matrix[u, v]
                    if alt < dist[v]:
                        dist[v] = alt
                        prev[v] = u
                        heapq.heappush(queue, (alt, v))

    # Creating the path
    path = []
    u = end
    while u is not None:
        path.append(u)
        u = prev[u]
    path.reverse()

    return path if dist[end] != np.inf else None

s_city = 'Rome'
e_city = 'Bolzano'

start_city = CITIES[CITIES['name'] == s_city].index[0]
end_city = CITIES[CITIES['name'] == e_city].index[0]

path_indices = dijkstra_with_angle_limit(start_city, end_city)
if path_indices is not None:
    path_cities = CITIES.iloc[path_indices]['name'].tolist()
    print(f"--------- Looking for path between {s_city} and {e_city} ---------")
    print("Best path:", " -> ".join(path_cities))
    print("Cost for best path: ", path_cost(path_indices))
    
else:
    print("There is not a path between the two cities!")

--------- Looking for path between Rome and Bolzano ---------
Best path: Rome -> Forlì -> Bolzano
Cost for best path:  520.5980314423081
